In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
RSEED=50

In [2]:
X_test = pd.read_csv('X_test.csv')

In [3]:
X_train = pd.read_csv('X_train.csv')

In [37]:
y_test = pd.read_csv('y_test.csv')
y_test.head()

,Unnamed: 0,y
0,2076,0
1,298,0
2,45,0
3,661,0
4,3877,0


In [6]:
y_train = pd.read_csv('y_train.csv')

### Importing the Model


In [8]:
from sklearn.ensemble import RandomForestClassifier


### Building the Model

In [9]:
model = RandomForestClassifier(n_estimators=100, 
                               random_state=RSEED, 
                               max_features = 'sqrt',
                               n_jobs=-1, verbose = 1)

### Fitting the Model

In [10]:
model.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   22.2s finished


RandomForestClassifier(n_jobs=-1, random_state=50, verbose=1)

### Testing Accuracy


In [55]:
y_pred = model.predict(X_test)

y_pred = pd.DataFrame(y_pred)
y_pred['1']

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.4s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    1.5s finished


KeyError: '1'

In [46]:
y_test

,Unnamed: 0,y
0,2076,0
1,298,0
2,45,0
3,661,0
4,3877,0
...,...,...
822,177,1
823,50,0
824,64,0
825,2928,0


In [54]:
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score

accuracy_score(y_test['y'], y_pred)

ValueError: Classification metrics can't handle a mix of binary and multiclass-multioutput targets

### Node count and Maximum depth of the tree

In [11]:
n_nodes = []
max_depths = []

for ind_tree in model.estimators_:
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

Average number of nodes 4177
Average maximum depth 41


## Assessing Random Forest Performance